In [1]:
import time
import os
import numpy as np
import matplotlib.pyplot as plt

#feature detection
from skimage.feature import SIFT, match_descriptors, plot_matches 
descriptor_extractor = SIFT() 

#ransac
from skimage.transform import AffineTransform 
from skimage.measure import ransac
from skimage.feature import plot_matches

#warping
from skimage.transform import warp

from skimage import io
import re #regex


In [2]:
import os.path as path
fixed_file = path.abspath(path.join(os.getcwd() ,"../..")) + '/pre/registered0/reg_concat_chAvg_sliceAvg.tif'
fixed = io.imread(fixed_file)

moving = io.imread('reg_concat_chAvg_sliceAvg.tif') 

moving_stack_ch0 = io.imread('reg_concat_ch0.tif')
moving_stack_ch1 = io.imread('reg_concat_ch1.tif')


In [3]:
#SIFT
descriptor_extractor.detect_and_extract(fixed) 
keypoints1 = descriptor_extractor.keypoints 
descriptors1 = descriptor_extractor.descriptors 

output_shape = fixed.shape 

descriptor_extractor.detect_and_extract(moving) 
keypoints2 = descriptor_extractor.keypoints
descriptors2 = descriptor_extractor.descriptors

matches12 = match_descriptors(descriptors1, descriptors2, cross_check=True, max_ratio = 0.7)


In [4]:
#ransac
src = keypoints2[matches12[:, 1]][:, ::-1]
dst = keypoints1[matches12[:, 0]][:, ::-1]

try:
    model_robust, inliers = ransac((src, dst), AffineTransform,
                                   min_samples=10, residual_threshold=5, max_trials=3000)
except ValueError:
    model_robust, inliers = ransac((src, dst), AffineTransform,
                                   min_samples=5, residual_threshold=5, max_trials=3000)
    err_str = ('ValueError for slice, necessary to lower `min_samples` to 5, may negatively impact warping')
    print(err_str)


In [5]:
#warp
try:
    warped_ = warp(moving, model_robust.inverse, preserve_range=True,
               output_shape=output_shape, cval=-1)
    warped_ = np.ma.array(warped_, mask=warped_==-1)
except AttributeError:
    warped_ = moving #no warping, use initial slice
    err_str = ('AttributeError for slice, slice not warped')
    print(err_str)


In [6]:
#write outputs

io.imsave('reg_concat_regToPre_sliceAvg_chAvg.tif', warped_.data)

ch0_reg_stack = [] #moving_stack_ch1
ch1_reg_stack = []

for i in range(np.shape(moving_stack_ch0)[0]):
    warped_ = warp(moving_stack_ch0[i], model_robust.inverse, preserve_range=True,
               output_shape=output_shape, cval=-1)
    warped_ = np.ma.array(warped_, mask=warped_==-1) 
    ch0_reg_stack.append(warped_.data) 

for i in range(np.shape(moving_stack_ch1)[0]):
    warped_ = warp(moving_stack_ch1[i], model_robust.inverse, preserve_range=True,
               output_shape=output_shape, cval=-1)
    warped_ = np.ma.array(warped_, mask=warped_==-1)
    ch1_reg_stack.append(warped_.data)

ch0_reg_stack = np.array(ch0_reg_stack, dtype=np.float32)
ch1_reg_stack = np.array(ch1_reg_stack, dtype=np.float32)

io.imsave('reg_concat_regToPre_ch0.tif', ch0_reg_stack)
io.imsave('reg_concat_regToPre_ch1.tif', ch1_reg_stack)